## [ Chapter 13 - Semantic Search with Dense Vectors ] 
## Setting up the outdoors dataset

We're going to use the Outdoors dataset for this chapter, and for a very important reason: the vocabulary and contexts in the outdoor question and answer __domain__ already have good coverage in the Transformer models we'll be using.

This is because the datasets that were used to train the model include sources that are likely to have similar subject matter.  Wikipedia was used to train bert-base-uncased (https://huggingface.co/bert-base-uncased#training-data) and, surprise! wikipedia has a section specifically on outdoors content: https://en.wikipedia.org/wiki/Outdoor

This is important, because if the words and their contexts haven't been seen before, the model will be less accurate.

Also, who doesn't enjoy playing around with a new dataset?! Data is search nerd candy.

In [1]:
import html
import json

from IPython.display import HTML, display

from aips import get_engine, set_engine
engine = get_engine()

## Listing 13.1

### Creating our Collection and Indexing the documents

In [2]:
#outdoors
![ ! -d 'outdoors' ] && git clone --depth=1 https://github.com/ai-powered-search/outdoors.git
! cd outdoors && git pull
! cd outdoors && cat outdoors.tgz.part* > outdoors.tgz
! cd outdoors && mkdir -p '../data/outdoors/' && tar -xvf outdoors.tgz -C '../data/outdoors/'

Already up to date.
README.md
concepts.pickle
._guesses.csv
guesses.csv
._guesses_all.json
guesses_all.json
outdoors_concepts.pickle
outdoors_embeddings.pickle
._outdoors_golden_answers.csv
outdoors_golden_answers.csv
._outdoors_golden_answers.xlsx
outdoors_golden_answers.xlsx
._outdoors_golden_answers_20210130.csv
outdoors_golden_answers_20210130.csv
outdoors_labels.pickle
outdoors_question_answering_contexts.json
outdoors_questionanswering_test_set.json
outdoors_questionanswering_train_set.json
._posts.csv
posts.csv
predicates.pickle
pull_aips_dependency.py
._question-answer-seed-contexts.csv
question-answer-seed-contexts.csv
question-answer-squad2-guesses.csv
._roberta-base-squad2-outdoors
roberta-base-squad2-outdoors/
roberta-base-squad2-outdoors/._tokenizer_config.json
roberta-base-squad2-outdoors/tokenizer_config.json
roberta-base-squad2-outdoors/._special_tokens_map.json
roberta-base-squad2-outdoors/special_tokens_map.json
roberta-base-squad2-outdoors/._config.json
roberta-base-

In [3]:
from aips.data_loaders.outdoors import load_dataframe

outdoors_collection = engine.create_collection("outdoors")
outdoors_dataframe = load_dataframe("data/outdoors/posts.csv")
outdoors_collection.write(outdoors_dataframe)
outdoors_dataframe.printSchema()

Wiping "outdoors" collection
Creating "outdoors" collection
Schema: {
  "mappings": {
    "properties": {
      "id": {
        "type": "text",
        "store": true
      },
      "post_type": {
        "type": "text",
        "store": true
      },
      "accepted_answer_id": {
        "type": "integer",
        "store": true
      },
      "parent_id": {
        "type": "integer",
        "store": true
      },
      "creation_date": {
        "type": "text",
        "store": true
      },
      "score": {
        "type": "integer",
        "store": true
      },
      "view_count": {
        "type": "integer",
        "store": true
      },
      "body": {
        "type": "text",
        "store": true,
        "fielddata": true
      },
      "owner_user_id": {
        "type": "text",
        "store": true
      },
      "title": {
        "type": "text",
        "store": true,
        "fielddata": true
      },
      "tags": {
        "type": "keyword",
        "store": true
     

## Listing 13.2

### Exploring data for a question regarding `climbing knots`

In [4]:
def get_posts_for_question(query):
    request = {"query": query,
               "limit": 1, 
               "query_fields": ["title", "body"],
               "return_fields": ["id"],
               "filters": [("post_type", "question"),
                           ("accepted_answer_id", "*")],
                           "log": True}
    id = outdoors_collection.search(**request)["docs"][0]["id"]
    request = {"query": id,
               "query_fields": ["id", "parent_id"],
               "limit": 3,
               "return_fields": ["id", "post_type", "title", "body", "parent_id", "accepted_answer_id"],
                           "log": True}
    return outdoors_collection.search(**request)["docs"]

In [5]:
get_posts_for_question("climbing knots")

Search Request:
{
  "query": {
    "bool": {
      "must": [
        {
          "query_string": {
            "query": "climbing knots",
            "boost": 0.454545454,
            "default_operator": "OR",
            "fields": [
              "title",
              "body"
            ]
          }
        }
      ],
      "filter": [
        {
          "term": {
            "post_type": "question"
          }
        },
        {
          "exists": {
            "field": "accepted_answer_id"
          }
        }
      ]
    }
  },
  "size": 1,
  "fields": [
    "id"
  ]
}
Search Request:
{
  "query": {
    "bool": {
      "must": [
        {
          "query_string": {
            "query": "18814",
            "boost": 0.454545454,
            "default_operator": "OR",
            "fields": [
              "id",
              "parent_id"
            ]
          }
        }
      ]
    }
  },
  "size": 3,
  "fields": [
    "id",
    "post_type",
    "title",
    "body",
    "par

[{'id': '18814',
  'accepted_answer_id': 18815,
  'creation_date': 1521697866173,
  'score': 4.280788,
  'view_count': 193,
  'body': 'One of the more common knots in climbing is the figure eight on a bight (being tied on a bight means that it is tied without using the ends. How exactly would one tie this knot?',
  'owner_user_id': '8794',
  'title': 'How to tie a figure eight on a bight?',
  'tags': ['climbing', 'knots'],
  'answer_count': 1,
  'post_type': 'question',
  'url': 'https://outdoors.stackexchange.com/questions/18814'},
 {'id': '18815',
  'parent_id': 18814,
  'creation_date': 1521697866173,
  'score': 0.45454547,
  'view_count': 0,
  'body': 'First you take the bight and bring the end down. Then you take the end and cross it over. Then bring it back around underneath the main strand. Finally take the end through the top loop. and there you have it. Before using you would want to dress it and tighten it down. If you want to see a video of the process for tying this knot, s

## Listing 13.3

### Querying our collection with a noun phrase

In [18]:
def display_questions(query, response, verbose=False):
    print(f"Query {query}:\n")
    if verbose:
        highlights = [response["highlighting"][k] for k in response["highlighting"].keys()]
    print("Ranked Questions:")
    for i, q in enumerate(response["docs"]):
        if verbose:
            print(json.dumps(q, indent="  "))
        if "title" in q.keys():
            id = f'<a href="{q["url"]}">{q["id"]}</a>'
            display(HTML(f'<strong>Question {id}: </strong>{q["title"]}'))
        if verbose:
            display(HTML("<strong>Body:</strong>" + html.unescape(str(highlights[i]["body"][0]))))
            display(HTML("<hr>"))

def search_questions(query, verbose=False):
    request = {"query": query,
               "query_fields": ["title", "body"],
               "limit": 5,
               "return_fields": ["id", "url", "post_type", "title",
                                 "body", "accepted_answer_id", "score"],
               "filters": [("post_type", "question")],
               "order_by": [("score", "desc"), ("title", "asc")]}
    response = outdoors_collection.search(**request)
    display_questions(query, response, verbose)

In [19]:
set_engine("opensearch")
outdoors_collection = get_engine().get_collection("outdoors")
search_questions("climbing knots")

Query climbing knots:

Ranked Questions:


In [21]:
outdoors_collection = get_engine().get_collection("outdoors")
search_questions("snow camping safety")

Query snow camping safety:

Ranked Questions:


## Listing 13.4

### Querying our collection with a question

In [22]:
search_questions("What is DEET?")

Query What is DEET?:

Ranked Questions:


Up next: [Introduction to Transformers](2.introduction-to-transformers.ipynb)